In [ ]:
import torch
import non_local_boxes
import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib.colors import hsv_to_rgb
import matplotlib.patches as mpatches

import matplotlib.lines as mlines

# Sugar coating for reloading
%matplotlib inline
%load_ext autoreload
%autoreload 2

# from IPython.display import set_matplotlib_formats
# set_matplotlib_formats('svg')   # in ordert to have unblurred pictures
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300

# Definitions

In [ ]:
matrix_to_tensor = non_local_boxes.utils.matrix_to_tensor
tensor_to_matrix = non_local_boxes.utils.tensor_to_matrix

# Boxes as 2x2x2x2 tensors
PR = matrix_to_tensor(non_local_boxes.utils.PR)
PRprime = matrix_to_tensor(non_local_boxes.utils.PRprime)
SR = matrix_to_tensor(non_local_boxes.utils.SR)
I = matrix_to_tensor(non_local_boxes.utils.I)

n = non_local_boxes.evaluate.nb_columns
W_BS09 = non_local_boxes.utils.W_BS09(n)

def WinCHSH(P): return float(non_local_boxes.evaluate.h_flat(P))                    # P is a 2x2x2x2 tensor
def WinCHSHprime(P): return float(non_local_boxes.evaluate.h_prime_flat(P))         # P is a 2x2x2x2 tensor
def BoxProduct(W,P,Q): return non_local_boxes.evaluate.R_tensor(W,P,Q)[:,:,:,:,0]   # P,Q are is a 2x2x2x2 tensor

def Catalan_number(k):
    return int(math.factorial(2*(k-1))/(math.factorial(k)*math.factorial(k-1)))

In [ ]:
def Compute_Coeff(P1, P2, P3, CHSHprime_value, CHSH_value):
    A = np.array([[WinCHSHprime(P1), WinCHSHprime(P2), WinCHSHprime(P3)],
                  [WinCHSH(P1), WinCHSH(P2), WinCHSH(P3)],
                  [1, 1, 1]])
    b = np.array([CHSHprime_value, CHSH_value, 1])
    return np.linalg.solve(A,b).tolist()

In [ ]:
def eta(P, x, y):
    return 2*P[0, x*y, x, y] + 2*P[1, (1+x*y)%2, x, y] -1

def BBP23_value(P):
    A = (eta(P,0,0)+eta(P,0,1)+eta(P,1,0)+eta(P,1,1))**2
    B = 2*eta(P,0,0)**2+4*eta(P, 1,0)*eta(P,0,1) + 2*eta(P, 1,1)**2
    return A+B

In [ ]:
def Compute_Orbits(P, W, max_depth):
    Orbit = [P]  # each all_orbits[k] is a list, which is the orbit of depth k+1
    for k in range(max_depth-1):
       Orbit.append( BoxProduct(W, Orbit[-1], P) )
    return Orbit

In [ ]:
def Compute_Orbit_BS09(P, W, max_depth):
    OrbitBS09 = [P]
    k = 1
    while 2**k<=max_depth:
        k += 1
        Q = OrbitBS09[-1]
        OrbitBS09.append( BoxProduct(W, Q, Q) )
    return OrbitBS09

In [ ]:
def E(Box, x, y):
    return Box[0,0,x,y] + Box[1,1,x,y] - Box[0,1,x,y] - Box[1,0,x,y]

def is_in_Q(CHSHprime_value,CHSH_value):
    alpha, beta, gamma = Compute_Coeff(PR, SR, I, CHSHprime_value=CHSHprime_value, CHSH_value=CHSH_value)
    Box = alpha*PR + beta*SR + gamma*I
        # Box is a 2x2x2x2 tensor
    coeff00 = np.arcsin(E(Box,0,0))
    coeff01 = np.arcsin(E(Box,0,1))
    coeff10 = np.arcsin(E(Box,1,0))
    coeff11 = np.arcsin(E(Box,1,1))
    return coeff00 + coeff01 + coeff10 - coeff11 <= np.pi

In [ ]:
def Draw_Orbit(P, P1, P1name, P2, P2name, P3, P3name, W, Wname, max_depth, details_BS09=False, precision_Q=0.0001, precision_coll_area=0.01):
    # P, P1, P2, P3 are 2x2x2x2 matrices

    # Draw the initial background triangle:
    x1, y1 = WinCHSHprime(P1), WinCHSH(P1)
    x2, y2 = WinCHSHprime(P2), WinCHSH(P2)
    x3, y3 = WinCHSHprime(P3), WinCHSH(P3)
    triangle = plt.Polygon(np.array([[x1, y1], [x2,y2], [x3, y3]]), color=hsv_to_rgb([0.475, 1, 0.9]))
    plt.gca().add_patch(triangle)

    BBLMTU_value = (3+math.sqrt(6))/6
    BBLMTU_rectangle = plt.Polygon(np.array([[0.5, 1.0], [0.5,BBLMTU_value], [1.5-BBLMTU_value, BBLMTU_value]]), color=hsv_to_rgb([0.475, 1, 0.5]))
    plt.gca().add_patch(BBLMTU_rectangle)

    # Draw the Quantum area
    CHSHprime_value=0.5
    CHSH_value=(2+np.sqrt(2))/4
    Quantum_color = 'pink'
    while CHSHprime_value<=0.75:
        if is_in_Q(CHSHprime_value, CHSH_value):
            plt.plot([CHSHprime_value, CHSHprime_value], [CHSH_value, 0.75], color=Quantum_color)
            CHSHprime_value+=precision_Q
        else:
            CHSH_value= max(CHSH_value-precision_Q, 0.75)
    plt.plot(0.75, 0.75,'o', markersize=0.7, color=Quantum_color)

    # Check if our coordinate system is good, i.e. if projected points aren't aligned:
    if (x1 - x3)*(y2-y3)==(x2-x3)*(y1-y3):
            print("WARNING: Error in the coordinate system.")
            return None
    

    x=0.5
    y=BBLMTU_value
    while y>0.75:
        alpha, beta, gamma = Compute_Coeff(PR, SR, I, CHSHprime_value=x, CHSH_value=y)
        P = alpha*PR + beta*SR + (1-alpha-beta)*I

        # Check if Orbit intersects the BBLMTU area:
        if details_BS09:
            OrbitBS09 = Compute_Orbit_BS09(P, W, max_depth)
            for Q in OrbitBS09:
                x = WinCHSHprime(Q)
                if x>=0.5:
                    y = WinCHSH(Q)
                    plt.plot(x, y, 'o', markersize=4, mfc='none', color='black')

        # Check if Orbit intersects the BBLMTU area:
        Orbits = Compute_Orbits(P, W, max_depth)
        for k in range(len(Orbits)):
            color = hsv_to_rgb([k/(7*(len(Orbits)-1)), 1, 1])
            for Q in Orbits[k]:  # Q is a tensor
                # Compute the abscissa and the ordinate:
                x = WinCHSHprime(Q)
                if x>=0.5:
                    y = WinCHSH(Q)
                    plt.plot(x, y, 'o', markersize=4, color=color)
        
        x+= precision_coll_area
        if x+y>3/2:
            x=0.5
            y-=precision_coll_area
                     
    
    
    
    plt.text(x1+0.001, y1, P1name, horizontalalignment='center',weight="bold")
    plt.text(x2, y2+0.007, P2name, verticalalignment='center',weight="bold")
    plt.text(x3, y3-0.007, P3name, verticalalignment='center',weight="bold")
    plt.title("Collapsing boxes, for computations until depth $k=$"+str(max_depth)+" with the wiring $\mathsf{W}=$"+Wname+" ")
    plt.xlabel("$\mathbb{P}($win at CHSH'$)$")
    plt.ylabel("$\mathbb{P}($win at CHSH$)$")
    handles = []
    handles.append( mpatches.Patch(color=hsv_to_rgb([0.475, 1, 0.5]), label='Collapsing $\mathcal{N\!S}$-boxes from [BBLMTU06].') )
    handles.append( mpatches.Patch(color=hsv_to_rgb([0.475, 1, 0.8]), label='Other $\mathcal{N\!S}$-boxes.') )
    handles.append( mpatches.Patch(color=Quantum_color, label='Quantum boxes.') )
    handles.append( mlines.Line2D([], [],color=hsv_to_rgb([0.0, 1, 1]), marker='o',ls='', label='Collapsing boxes found in [BS09].') )
    handles.append( mlines.Line2D([], [],color=hsv_to_rgb([1/14, 1, 1]), marker='o',ls='', label='Collapsing boxes found in [BBP23].') )
    handles.append(  mlines.Line2D([], [],color=hsv_to_rgb([1/7, 1, 1]), marker='o',ls='', label='Our new collapsing boxes.') )
    plt.legend(handles=handles, loc='upper right', fontsize = 9)
    plt.show()


# Compute the Orbit

In [ ]:
# alpha = 0.47
# beta = 0.51
alpha, beta, gamma = Compute_Coeff(PR, SR, I, CHSHprime_value=0.627, CHSH_value=0.862)
P = alpha*PR + beta*SR + (1-alpha-beta)*I

Orbits = Draw_Orbit(
    P = P,
    P1 = PR,
    P1name = "PR",
    P2 = SR,
    P2name = "SR",
    P3 = (I+PR)/2,
    P3name = "(I+PR)/2",
    W = W_BS09,
    Wname = "$\mathsf{W}_{bs}$",
    max_depth = 5, 
    details_BS09=True,
    precision_Q=0.001
)

In [ ]:
alpha = 0.47
beta = 0.51
P = alpha*PR + beta*SR + (1-alpha-beta)*I

Orbits = Draw_Orbit(
    P = P,
    P1 = PR,
    P1name = "PR",
    P2 = SR,
    P2name = "SR",
    P3 = (I+PR)/2,
    P3name = "(I+PR)/2",
    W = non_local_boxes.utils.W_FWW09(n),
    Wname = "$W_{\oplus}$",
    max_depth = 12,
    precision_Q=0.0001
)

# Other Stuff

In [ ]:
W = W_BS09
max_depth = 6

print("BBLMTU value: ", (3+math.sqrt(6))/6)
print("------")

Orbits = Compute_Orbits(P, W, max_depth)
for k in range(len(Orbits)):
    y = 0
    for Q in Orbits[k]:
        y = max(WinCHSH(Q), y)
    print("For k=",k+1,", CHSH value = ",y)

print("------")
print("ORBIT BS09")
OrbitBS09 = Compute_Orbit_BS09(P, W, 2**max_depth)
for Q in OrbitBS09:
    y = WinCHSH(Q)
    print(y)

In [ ]:
x = WinCHSHprime(P)
y = WinCHSH(P)
print(x,y)
print(BBP23_value(P))

In [ ]:
OrbitBS09 = Compute_Orbit_BS09(P, W, 2**max_depth)
for Q in OrbitBS09:
    print("---")
    OrbitBS09_Q = Compute_Orbit_BS09(Q, W, 2**max_depth)
    for R in OrbitBS09_Q:
        y = WinCHSH(R)
        print(y)